In [14]:
import numpy as np
import os
import matplotlib.pylab as plt
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import tf_keras

In [2]:
BATCH_SIZE = 32
IMAGE_SIZE = 224
CHANNELS=3
EPOCHS=30

In [3]:
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    "output",
    shuffle=True,
    image_size=(IMAGE_SIZE,IMAGE_SIZE),
    batch_size=BATCH_SIZE
)

Found 7390 files belonging to 2 classes.


In [4]:
class_names = dataset.class_names
class_names

['cat', 'dog']

In [5]:
data_dir = "output"
import pathlib
data_dir = pathlib.Path(data_dir)
data_dir

WindowsPath('output')

In [6]:
animal_images_dict = {
    'cat': list(data_dir.glob('cat/*')),
    'dog': list(data_dir.glob('dog/*'))}
animals_labels_dict = {
    'cat': 0,
    'dog': 1,}

In [7]:
X, y = [], []
import cv2
for animal, images in animal_images_dict.items():
    for image in images:
        image_path = os.path.normcase(image)
        
        # Görüntünün varlığını ve formatını kontrol et
        
        
        # Görüntüyü oku
        img = cv2.imread(image_path)
        if img is None:
            print(f"Okuma başarısız: {image_path}")
            continue
        
        resized_img = cv2.resize(img,(224,224))
        X.append(resized_img)
        y.append(animals_labels_dict[animal])


Okuma başarısız: output\cat\abyssinian_34.jpg
Okuma başarısız: output\cat\egyptian_mau_139.jpg
Okuma başarısız: output\cat\egyptian_mau_145.jpg
Okuma başarısız: output\cat\egyptian_mau_167.jpg
Okuma başarısız: output\cat\egyptian_mau_177.jpg
Okuma başarısız: output\cat\egyptian_mau_191.jpg


In [8]:
X = np.array(X)
y = np.array(y)

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42) # random state kaldırılabilir
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)
# ilk satırda x_temp içine 0.3 ünü atıyoruz sonrasında bunun yarısını x_test diger yarısını x_val içerisine alıyoruz yani 0.15 olur

In [11]:
X_train_scaled = X_train / 255 # piksel degerlerini 0 - 1 arasına almak için kullanılır
X_test_scaled = X_test / 255
X_val_scaled = X_val / 225

In [12]:
img_width, img_height = 224, 224

In [13]:
feature_extractor_model = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"

pretrained_model_without_top_layer = hub.KerasLayer(
    feature_extractor_model, input_shape=(224, 224, 3), trainable=False)

In [15]:
num_of_categories = 2
model = tf_keras.Sequential([
      pretrained_model_without_top_layer,
        tf_keras.layers.Dense(num_of_categories)
    ])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 1280)              2257984   
                                                                 
 dense (Dense)               (None, 2)                 2562      
                                                                 
Total params: 2260546 (8.62 MB)
Trainable params: 2562 (10.01 KB)
Non-trainable params: 2257984 (8.61 MB)
_________________________________________________________________


In [16]:
model.compile(
  optimizer="adam",
  loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
  metrics=['acc'])



In [17]:
model.fit(X_train_scaled, y_train, validation_data=(X_val_scaled, y_val), epochs=2 )

MemoryError: Unable to allocate 5.80 GiB for an array with shape (5168, 224, 224, 3) and data type float64